In [ ]:
import matplotlib.pyplot as plt

# Sample data: Dictionary where keys are person IDs and values are lists of time intervals
data = {
    1: [(10, 20), (30, 50), (60, 80)],  # For person ID 1, time intervals are (10, 20), (30, 50), and (60, 80)
    2: [(15, 25), (40, 60), (70, 90)],  # For person ID 2, time intervals are (15, 25), (40, 60), and (70, 90)
    3: [(5, 15), (35, 55), (65, 85)]    # For person ID 3, time intervals are (5, 15), (35, 55), and (65, 85)
}

# Plotting
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

for person_id, intervals in data.items():
    for interval in intervals:
        start, end = interval
        plt.barh(person_id, width=end-start, left=start, height=0.1, color='#4338CA')

plt.xlabel('Time')
plt.ylabel('Person ID')
plt.title('Time Intervals for Person IDs')
plt.yticks(list(data.keys()), ['Person {}'.format(pid) for pid in data.keys()])

plt.show()


In [ ]:
import os
import json
import pandas as pd

userlog_path = "UserLog"
data = []
pd = {}
vd = {}

# for user_folder in os.listdir(userlog_path):
for folder_number in range(1, 13):
    user_folder = f'P{folder_number}'
    print(f'Processing {user_folder}')
    user_data = {}
    user_folder_path = os.path.join(userlog_path, user_folder)
    if os.path.isdir(user_folder_path):
        folder_number = int(user_folder[1:])
        # if folder_number % 2 == 0:
        for subdir, _, files in os.walk(user_folder_path):
            for file in files:
                file_path = os.path.join(subdir, file)
                if file == 'onboarding.json':
                    pass
                # if not file.lower().startswith('video1') and not file.lower().startswith('video2'): continue
                else:
                    video_data = {}
                    vd_data = {}
                    with open(file_path, 'r') as f:
                        bullet_points_data = json.load(f)
                    video_data['pauseCount'] = bullet_points_data['pauseCount']
                    video_data['forwardCount'] = bullet_points_data['forwardCount']
                    video_data['reverseCount'] = bullet_points_data['reverseCount']
                    video_data['point_count'] = len(bullet_points_data['editHistory'])
                    summary_t = bullet_points_data['summary_t']
                    summary_p = bullet_points_data['summary_p']
                    note_points = [{'point': bpd['point'], 'time_taken': bpd['note_taking_time']} for bpd in bullet_points_data['editHistory']]
                    vd_data['p_id'] = user_folder
                    vd_data['note_points'] = note_points
                    vd_data['summary_p'] = summary_p
                    vd_data['summary_t'] = summary_t
                    if folder_number % 2 == 0:
                        if file.lower().startswith('video1'): 
                            user_data['Baseline'] = video_data
                            vd_data['micronote'] = False
                            if 'video1' not in vd:
                                vd['video1'] = []
                            vd['video1'].append(vd_data)
                        elif file.lower().startswith('video2'): 
                            user_data['NoTeeline'] = video_data
                            vd_data['micronote'] = True
                            if 'video2' not in vd:
                                vd['video2'] = []
                            vd['video2'].append(vd_data)
                    else:
                        if file.lower().startswith('video1'): 
                            user_data['NoTeeline'] = video_data
                            vd_data['micronote'] = True
                            if 'video1' not in vd:
                                vd['video1'] = []
                            vd['video1'].append(vd_data)
                        elif file.lower().startswith('video2'): 
                            user_data['Baseline'] = video_data
                            vd_data['micronote'] = False
                            if 'video2' not in vd:
                                vd['video2'] = []
                            vd['video2'].append(vd_data)
    data.append(user_data)

# print('############ COUNT DATA ############')
# for i, d in enumerate(data):
#     print('----------------------------------------')
#     print(f'User {i+1}')
#     print(d)

print('\n############ VIDEO DATA ############')
for video in vd:
    print(f'--------------\n{video} data...\n--------------')
    for v in vd[video]:
        print('p_id\n----')
        print(f"{v['p_id']}")
        print('micronote\n---------')
        print(f"{v['micronote']}")
        print('summary_t\n---------')
        print(f"{v['summary_t']}")
        print('summary_p\n---------')
        print(f"{v['summary_p']}")
        print(f"Notes:\n------\n{v['note_points']}")
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


chosen_cmap = matplotlib.cm.get_cmap('Paired')
# color_array = [chosen_cmap(i) for i in np.linspace(0, 1, 6)]
# color_hex_array = [matplotlib.colors.to_hex(color) for color in color_array]

# Lists to hold the plot data
labels = []
noteeline_pause = []
noteeline_star_pause = []
noteeline_forward = []
noteeline_star_forward = []
noteeline_reverse = []
noteeline_star_reverse = []

# Populate the lists with data
for user, activities in enumerate(data):
    print(activities)
    labels.append(f'P{user+1}')
    noteeline_pause.append(activities['NoTeeline']['pauseCount'])
    noteeline_star_pause.append(activities['Baseline']['pauseCount'])
    noteeline_forward.append(activities['NoTeeline']['forwardCount'])
    noteeline_star_forward.append(activities['Baseline']['forwardCount'])
    noteeline_reverse.append(activities['NoTeeline']['reverseCount'])
    noteeline_star_reverse.append(activities['Baseline']['reverseCount'])

# Width of the bars
barWidth = 0.3

# Set position of bar on X axis
r1 = np.arange(len(labels))
r2 = [x + barWidth for x in r1]

# Make the plot
plt.figure(figsize=(10, 6))

# Stacked bar for 'noteeline'
bar1 = plt.bar(r1, noteeline_pause, color=chosen_cmap(1), width=barWidth, edgecolor='white', label='Pause (NoTeeline)')
bar2 = plt.bar(r1, noteeline_forward, bottom=noteeline_pause, color=chosen_cmap(3), width=barWidth, edgecolor='white', label='Seek Forward (NoTeeline)')
bar3 = plt.bar(r1, noteeline_reverse, bottom=[i+j for i,j in zip(noteeline_pause, noteeline_forward)], color=chosen_cmap(5), width=barWidth, edgecolor='white', label='Seek Backward (NoTeeline)')

# Stacked bar for 'noteeline*'
bar4 =plt.bar(r2, noteeline_star_pause, color=chosen_cmap(0), width=barWidth, edgecolor='white', label='Pause (Baseline)')
bar5 = plt.bar(r2, noteeline_star_forward, bottom=noteeline_star_pause, color=chosen_cmap(2), width=barWidth, edgecolor='white', label='Seek Forward (Baseline)')
bar6 = plt.bar(r2, noteeline_star_reverse, bottom=[i+j for i,j in zip(noteeline_star_pause, noteeline_star_forward)], color=chosen_cmap(4), width=barWidth, edgecolor='white', label='Seek Backword (Baseline)')

# Add xticks on the middle of the group bars
# plt.xlabel('User', fontweight='bold')
plt.xticks([r + barWidth/2 for r in range(len(labels))], labels)
plt.legend(loc="lower left", ncol=2, bbox_to_anchor=(0.18, -0.3), frameon=False) # https://stackoverflow.com/a/54870844
plt.yticks([])  

# Create legend & Show graphic
# plt.legend()
plt.box(False)
plt.savefig('video_log_dist.pdf', bbox_inches="tight")
plt.show()


In [ ]:
import matplotlib.pyplot as plt


noteeline_cnt = []
baseline_cnt = []
x = []
labels = []

# Populate the lists with data
for user, activities in enumerate(data):
    # print(activities)
    labels.append(f'P{user+1}')
    x.append(user+1)
    noteeline_cnt.append(activities['NoTeeline']['point_count'])
    baseline_cnt.append(activities['Baseline']['point_count'])
plt.figure(figsize=(10, 6))

# Plot the first line with x and y1 values
plt.plot(x, noteeline_cnt, label='NoTeeline', marker='o', color=chosen_cmap(3))

# Plot the second line with x and y2 values
plt.plot(x, baseline_cnt, label='Baseline', marker='o', color=chosen_cmap(9))


# Adding the legend, which uses the labels defined in the plt.plot() method
plt.legend(loc="lower left", bbox_to_anchor=(0.3, -0.2), ncol = 2, frameon=False)

plt.xticks([r+1 for r in range(len(labels))], labels)
# plt.xlabel('X-axis Label')
plt.ylabel('# of note points')
plt.box(False)
plt.savefig('note_count_dist.pdf', bbox_inches="tight")
# Finally, show the plot
plt.show()
